In [1]:
# ruff: noqa: N802, N803, N806, N815, N816
import os

import numpy as np
from scipy import signal

# Simple utilities for displaying generated code in the notebook
from utils import cleanup, display_text

import archimedes as arc

THEME = os.environ.get("ARCHIMEDES_THEME", "dark")
arc.theme.set_theme(THEME)

# Platform-specific deployment templates

So far in this tutorial we have seen how Archimedes can leverage CasADi's code generator to translate pure Python functions to efficient C code.
We then explored templated "driver code" generation and saw how auto-generated code can be customized using protected regions and template modifications.

The final step on the road to hardware deployment is simply to change the driver template from the plain-C `main.c` generator to a platform-specific template.

First, let's recap our example function: the IIR filter.

In [2]:
# Optionally give descriptive names for return values (these don't need
# to match the variable names)
@arc.compile(return_names=["u_hist", "y_hist"])
def iir_filter(u, b, a, u_prev, y_prev):
    # Update input history
    u_prev[1:] = u_prev[:-1]
    u_prev[0] = u

    # Compute output using the direct II transposed structure
    y = (np.dot(b, u_prev) - np.dot(a[1:], y_prev[: len(a) - 1])) / a[0]

    # Update output history
    y_prev[1:] = y_prev[:-1]
    y_prev[0] = y

    return u_prev, y_prev

In [4]:
# Design a simple IIR filter with SciPy
dt = 0.01  # Sampling time [seconds]
Wn = 10  # Cutoff frequency [Hz]
order = 4
b, a = signal.butter(order, Wn, "low", analog=False, fs=1 / dt)

# Create "template" arguments for type inference
u = 1.0
u_prev = np.zeros(len(b))
y_prev = np.zeros(len(a) - 1)
args = (u, b, a, u_prev, y_prev)

As a simple example of hardware deployment, we'll target a garden-variety Arduino development board.
The default template for Arduino requires that the sample rate in seconds be provided and sets up an ISR flag for relatively accurate loop timing.

In [ ]:
cleanup()  # Clean up any previous generated code

driver_config = {
    "sample_rate": dt,
    "output_path": "iir_driver.ino",
}

arc.codegen(
    iir_filter, "iir_filter.c", args, driver="arduino", driver_config=driver_config
)

In [6]:
with open(driver_config["output_path"], "r") as f:
    sketch = f.read()

display_text(sketch)

```c
#include <Arduino.h>
#include <TimerOne.h>
#include "iir_filter.h"

// PROTECTED-REGION-START: imports
// ... User-defined imports and includes
// PROTECTED-REGION-END

// Sampling rate: 100 Hz
const unsigned long SAMPLE_RATE_US = 10000;

// Allocate memory for inputs and outputs
double u = 1.0;
double b[5] = {0.004824343357716229, 0.019297373430864916, 0.028946060146297373, 0.019297373430864916, 0.004824343357716229};
double a[5] = {1.0, -2.369513007182038, 2.313988414415881, -1.054665405878568, 0.18737949236818502};
double u_prev[5] = {0.0, 0.0, 0.0, 0.0, 0.0};
double y_prev[4] = {0.0, 0.0, 0.0, 0.0};

double u_hist[5] = {0};
double y_hist[4] = {0};

// Prepare pointers to inputs, outputs, and work arrays
const double* arg[iir_filter_SZ_ARG] = {0};
double* res[iir_filter_SZ_RES] = {0};
long long int iw[iir_filter_SZ_IW];
double w[iir_filter_SZ_W];

// Flag for interrupt timer
volatile bool control_loop_flag = false;

// PROTECTED-REGION-START: allocation
// ... User-defined memory allocation and function declaration
// PROTECTED-REGION-END

// Timer interrupt handler
void timerInterrupt() {
    // PROTECTED-REGION-START: interrupt
    // Set flag for main loop to run control function
    control_loop_flag = true;
    // PROTECTED-REGION-END
}

void setup(){
    // Set up input and output pointers
    arg[0] = &u;
    arg[1] = b;
    arg[2] = a;
    arg[3] = u_prev;
    arg[4] = y_prev;

    res[0] = u_hist;
    res[1] = y_hist;

    // PROTECTED-REGION-START: setup
    // ... User-defined setup code
    Serial.begin(9600);
    // PROTECTED-REGION-END

    // Initialize Timer1 for interrupts at 100 Hz
    Timer1.initialize(SAMPLE_RATE_US);
    Timer1.attachInterrupt(timerInterrupt);
}

void loop() {
    // Check if control loop should run (set by timer interrupt)
    if (control_loop_flag) {
        control_loop_flag = false;
        
        // PROTECTED-REGION-START: control_loop
        // ... User-defined timed code
        iir_filter(arg, res, iw, w, 0);
        // PROTECTED-REGION-END
    }
    
    // PROTECTED-REGION-START: loop
    // ... User-defined non-time-critical tasks
    delay(10);
    // PROTECTED-REGION-END
}
```

Of course, just as for the plain-C template, most applications will require some customization of this "driver" code.
You might need to use a different timing pattern, declare pin configurations, interact with sensors and actuators, add communication, or include any other boilerplate your project needs.

For simple modifications, you can once again edit the protected regions and your changes will be preserved when the code is re-generated.
For more extensive or structural changes, you can simply copy the default template and modify it freely.

## Conclusion

In this tutorial we explored the mechanics of the Archimedes workflow for deploying your algorithms to embedded controllers.

The basic underpinning for this process is CasADi's code generation tool, which generates a highly efficient C code representation of the computational graph.
With this we can automatically translate most functions compatible with the [`compile`](#archimedes.compile) decorator to C code (some notable exceptions include calls to plugin solvers like IPOPT and SUNDIALS).

However, the auto-generated function has a generic signature that requires specific memory configuration.
To manage this without manual intervention, Archimedes also provides a templated "driver" code generation system.
This process uses standard Jinja2 templates to declare and initialize all the variables used by your functions.
In addition, the templating system provides "protected regions" in which you can modify the auto-generated code and have your changes preserved.

The combination of customizable templates with the Python-to-C translation enables a powerful workflow where a project- and platform-specific template can be used to configure low-level computation like peripheral I/O and communication while the high-level control logic can be developed, simulated, tested, and automatically deployed into your driver code.

In this workflow:

1. Develop and test your algorithm in Python using Archimedes and NumPy
2. Use `codegen` to generate the core algorithm and driver code
3. Make hardware-specific customizations in the protected regions or driver template
4. As you refine your algorithm in Python, regenerate the C code while preserving your customizations
5. Compile and deploy to your target hardware

This development cycle maintains a clean separation between algorithm development (in Python) and hardware-specific implementation details (in the protected regions of your C code).

### Where to Go From Here

As mentioned, this hardware deployment workflow is an early proof of concept, but it is central to the Archimedes roadmap. Planned developments include:

- Support for more platforms (e.g. STM32, ESP32, Raspberry Pi)
- Workflows for hardware-in-the-loop (HIL) testing
- Performance optimizations like fixed-point arithmetic
- Support for real-time constraints and multirate scheduling (RTOS)
- Integration with static analysis tools for memory and executing time

**If you're using Archimedes for hardware control and have questions, comments, or requests, please don't hesitate to reach out!**